In [47]:
import alpaca_trade_api as tradeapi
import threading
import time
import os
import datetime as dt
import pandas as pd

LE = []
SE = []


class PortfolioManager():
    def __init__(self):
        self.api = tradeapi.REST(base_url = ep, key_id = api_key, secret_key=secret_key)
        self.r_positions = {}
        self.ETB = self.get_tickers(10,500) #Min, Max

    def format_percent(self, num):
        if(str(num)[-1] == "%"):
            return float(num[:-1]) / 100
        else:
            return float(num)

    def clear_orders(self):
        try:
            self.api.cancel_all_orders()
            print("All open orders cancelled.")
        except Exception as e:
            print(f"Error: {str(e)}")

    def add_items(self, data):
        ''' Expects a list of lists containing two items: symbol and position qty/pct
        '''
        for row in data:
            self.r_positions[row[0]] = [row[1], 0]

    def percent_rebalance(self, order_style, timeout=60):
        print(f"Desired positions: ")
        for sym in self.r_positions:
            print(f"{sym} - {self.r_positions.get(sym)[0]} of portfolio.")
        print()

        positions = self.api.list_positions()
        account = self.api.get_account()
        portfolio_val = float(account.portfolio_value)
        for sym in self.r_positions:
            try:
                price = self.api.get_barset(sym, "minute", 1)[sym][0].c
                self.r_positions[sym][0] = int(
                    self.format_percent(
                        self.r_positions.get(sym)[0]) * portfolio_val / price)
            except:
                print('Skipped {sym}')
                continue

        print(f"Current positions: ")
        for position in positions:
            print(
                f"{position.symbol} - {round(float(position.market_value) / portfolio_val * 100, 2)}% of portfolio.")
        print()

        self.clear_orders()

        print("Clearing extraneous positions.")
        for position in positions:
            if(self.r_positions.get(position.symbol)):
                self.r_positions.get(position.symbol)[1] = int(position.qty)
            else:
                self.send_basic_order(
                    position.symbol, position.qty, ("buy", "sell")[
                        position.side == "long"])
        print()

        if(order_style == "send"):
            for sym in self.r_positions:
                qty = self.r_positions.get(
                    sym)[0] - self.r_positions.get(sym)[1]
                self.send_basic_order(sym, qty, ("buy", "sell")[qty < 0])
        elif(order_style == "timeout"):
            threads = []
            for i, sym in enumerate(self.r_positions):
                qty = self.r_positions.get(
                    sym)[0] - self.r_positions.get(sym)[1]
                threads.append(
                    threading.Thread(
                        target=self.timeout_execution, args=(
                            sym, qty, ("buy", "sell")[
                                qty < 0], self.r_positions.get(sym)[0], timeout)))
                threads[i].start()

            for i in range(len(threads)):
                threads[i].join()
        elif(order_style == "block"):
            threads = []
            for i, sym in enumerate(self.r_positions):
                qty = self.r_positions.get(
                    sym)[0] - self.r_positions.get(sym)[1]
                threads.append(
                    threading.Thread(
                        target=self.confirm_full_execution, args=(
                            sym, qty, ("buy", "sell")[
                                qty < 0], self.r_positions.get(sym)[0])))
                threads[i].start()

            for i in range(len(threads)):
                threads[i].join()

    def rebalance(self, order_style, timeout=60):
        print(f"Desired positions: ")
        for sym in self.r_positions:
            print(f"{sym} - {self.r_positions.get(sym)[0]} shares.")
        print("\n")

        positions = self.api.list_positions()

        print(f"Current positions: ")
        for position in positions:
            print(f"{position.symbol} - {position.qty} shares owned.")
        print()

        self.clear_orders()

        print("Clearing extraneous positions.")
        for position in positions:
            if(self.r_positions.get(position.symbol)):
                self.r_positions[position.symbol][1] = int(position.qty)
            else:
                self.send_basic_order(
                    position.symbol, position.qty, ("buy", "sell")[
                        position.side == "long"])
        print()

        if(order_style == "send"):
            for sym in self.r_positions:
                qty = int(self.r_positions.get(sym)[
                          0]) - self.r_positions.get(sym)[1]
                self.send_basic_order(sym, qty, ("buy", "sell")[qty < 0])
        elif(order_style == "timeout"):
            threads = []
            for i, sym in enumerate(self.r_positions):
                qty = int(self.r_positions.get(sym)[
                          0]) - self.r_positions.get(sym)[1]
                threads.append(
                    threading.Thread(
                        target=self.timeout_execution, args=(
                            sym, qty, ("buy", "sell")[
                                qty < 0], self.r_positions.get(sym)[0], timeout)))
                threads[i].start()

            for i in range(len(threads)):
                threads[i].join()
        elif(order_style == "block"):
            threads = []
            for i, sym in enumerate(self.r_positions):
                qty = int(self.r_positions.get(sym)[
                          0]) - self.r_positions.get(sym)[1]
                threads.append(
                    threading.Thread(
                        target=self.confirm_full_execution, args=(
                            sym, qty, ("buy", "sell")[
                                qty < 0], self.r_positions.get(sym)[0])))
                threads[i].start()

            for i in range(len(threads)):
                threads[i].join()

    def send_basic_order(self, sym, qty, side):
        qty = int(qty)
        if(qty == 0):
            return
        q2 = 0
        try:
            position = self.api.get_position(sym)
            curr_pos = int(position.qty)
            if((curr_pos + qty > 0) != (curr_pos > 0)):
                q2 = curr_pos
                qty = curr_pos + qty
        except BaseException:
            pass
        try:
            if q2 != 0:
                self.api.submit_order(sym, abs(q2), side, "market", "gtc")
                try:
                    self.api.submit_order(sym, abs(qty), side, "market", "gtc")
                except Exception as e:
                    print(
                        f"Error: {str(e)}. Order of | {abs(qty) + abs(q2)} {sym} {side} | partially sent ({abs(q2)} shares sent).")
                    return False
            else:
                self.api.submit_order(sym, abs(qty), side, "market", "gtc")
            print(f"Order of | {abs(qty) + abs(q2)} {sym} {side} | submitted.")
            return True
        except Exception as e:
            print(
                f"Error: {str(e)}. Order of | {abs(qty) + abs(q2)} {sym} {side} | not sent.")
            return False

    def confirm_full_execution(self, sym, qty, side, expected_qty):
        sent = self.send_basic_order(sym, qty, side)
        if(not sent):
            return

        executed = False
        while(not executed):
            try:
                position = self.api.get_position(sym)
                if int(position.qty) == int(expected_qty):
                    executed = True
                else:
                    print(f"Waiting on execution for {sym}...")
                    time.sleep(20)
            except BaseException:
                print(f"Waiting on execution for {sym}...")
                time.sleep(20)
        print(
            f"Order of | {abs(qty)} {sym} {side} | completed.  Position is now {expected_qty} {sym}.")

    def timeout_execution(self, sym, qty, side, expected_qty, timeout):
        sent = self.send_basic_order(sym, qty, side)
        if(not sent):
            return
        output = []
        executed = False
        timer = threading.Thread(
            target=self.set_timeout, args=(
                timeout, output))
        timer.start()
        while(not executed):
            if(len(output) == 0):
                try:
                    position = self.api.get_position(sym)
                    if int(position.qty) == int(expected_qty):
                        executed = True
                    else:
                        print(f"Waiting on execution for {sym}...")
                        time.sleep(20)
                except BaseException:
                    print(f"Waiting on execution for {sym}...")
                    time.sleep(20)
            else:
                timer.join()
                try:
                    position = self.api.get_position(sym)
                    curr_qty = position.qty
                except BaseException:
                    curr_qty = 0
                print(
                    f"Process timeout at {timeout} seconds: order of | {abs(qty)} {sym} {side} | not completed. Position is currently {curr_qty} {sym}.")
                return
        print(
            f"Order of | {abs(qty)} {sym} {side} | completed.  Position is now {expected_qty} {sym}.")

    def set_timeout(self, timeout, output):
        time.sleep(timeout)
        output.append(True)
        
        
    '''Zach Additions ------------ 1.0'''
    def format_holdings(self,longs=[],shorts=[]):
        '''Returns proper format for PF Weights'''

        #longs = ['SPY','GLD','AMZN']
        #shorts = ['AAPL','TLT','XOM']

        #account = self.api.get_account()
        #portfolio_val = float(account.portfolio_value)
        total = len(longs) + len(shorts)
        if total != 0:
            qty = float(1.0 / (len(longs) + len(shorts)))
        else:
            print('ERROR -- Divide by zero, ensure Longs or Shorts has items')
        
        arg = []
        if len(longs) >= 0:
            for l in longs:
                arg.append([l,qty])
            #LE = [[long,qty] for long in longs]
        
        if len(shorts) > 0:
            for s in shorts:
                arg.append([s,-qty])

        return arg
    
    def parse(self,path=None):
        #Vectorized operations
        
        global LE, SE, CLOSE_DICT
        
        old_long = LE
        #old_short = SE
        
        #new_long = set()
        #new_short = set()
        
        #new_lx = set()
        #new_sx = set()
        
        DF = pd.DataFrame({'RA_T':[-1,3,5],'SYMBOL':['AAPL','INTC','AMD'],'P_Change':[1.5,2.5,3.5]})  #TEMPORARY !!!
        if path is not None:
            DF = pd.read_csv(path,header=None)
            DF.columns = ['Exchange','SYMBOL','RA_Y','RA_T','Close','P_Change']
            #DF = pd.read_csv("C:\\Users\\Administrator\\Downloads\\rankanalysis_05-05-2020.csv",header=None)
        
        #LE's
        strong = DF[DF['RA_T'] >= 4]
        mu = DF['P_Change'].mean()
        sig = DF['P_Change'].std()
        no_out = strong[strong['P_Change'] <= (mu+sig)]
        
        #SE's 
        weak = DF[DF['RA_T'] <= -4]
        #short_new = weak[weak['P_Change'] <= (mu+sig)]
        
        LE = no_out.SYMBOL.to_list()
        LX = [i for i in old_long if i not in LE]
        
        SE = weak.SYMBOL.to_list()
        
        return LE, SE
    
    def run(self,path=None):
        #from portfolio_manager import PortfolioManager
        ep = 'https://paper-api.alpaca.markets'
        api_key = 'PK4LB1H4BH7YUF4XOODN'
        secret_key = '7EuTmI/C/7FwtNkEzqzFD7yiVMqaI16XfuerzHgR'

        os.environ['APCA_API_KEY_ID'] = api_key
        os.environ['APCA_API_SECRET_KEY'] = secret_key

        manager = PortfolioManager()
        
        
        now = dt.datetime.now()
        open = False
        
        if now.hour == 8 and now.minute < 30:
            print(f'Algorithm Waiting for Market Open...')
            if not warm_up:
                LE, SE = self.parse(path)
                print(f'Parsed LE {len(LE)} and SE {len(SE)}')
                
                arg = self.format_holdings(LE,SE)
                manager.add_items(arg)
                
                if len(LE) > 0:
                    warm_up = True
            
        elif now.hour == 8 and now.minute == 30:
            print('Algorithm Open for Day')
            open = True     
            
        if open:
            manager.percent_rebalance('timeout')
            return 0
        return -1
            

    def liquidate_all(self):
        '''Liquidate ALL positions and orders'''
        orders = self.api.list_orders(status='open')
        positions = self.api.list_positions()
        
        rejected = {}
        if orders or positions:
            if positions:
                print(positions)

            if orders:
                print("Canceling open orders:")
                print([o.id for o in orders])
                result = [self.api.cancel_order(o.id) for o in orders]
                print(result)

            closed = []
            for p in positions:
                side = 'sell'
                
                if int(p.qty) < 0:
                    #If Short, CLOSE W Buy (else, still closing w sell)
                    p.qty = abs(int(p.qty))
                    side = 'buy'
                    
                closed.append(
                    self.api.submit_order(p.symbol, qty=p.qty, side=side, type="market", time_in_force="day")
                    )

            if closed:
                print("Submitted Orders:", closed)
            
            
            for o in closed:
                status = self.api.get_order(o.id)
                if status.status == 'rejected':
                    print("ORDER FAILED: Your Order was Rejected!!!")
                    rejected[o.oid] = status.status
                    
        return f'{len(rejected)} Orders Rejected.'
    
    
    def get_tickers(self,min_price=5,max_price=500,etb=True):
        print('Getting current ticker data...')
        tickers = self.api.polygon.all_tickers() #ALLLLLL SYMBOLS ! 
        print('All symbols loaded')
        assets = self.api.list_assets()
        if etb: #FILTER TRADABLE SYMBOLS; easy to borrow if switch True
            symbols = [asset.symbol for asset in assets if asset.tradable and asset.easy_to_borrow]
        else:
            symbols = [asset.symbol for asset in assets if asset.tradable] 
        #print(tickers)
        sort_lst =  sorted([ticker for ticker in tickers if ( #JUST return ticker if you want full JSON object
            ticker.ticker in symbols and
            ticker.lastTrade['p'] >= min_price and
            ticker.lastTrade['p'] <= max_price and
            ticker.prevDay['v'] * ticker.lastTrade['p'] > 500000 #and
            #ticker.todaysChangePerc >= 3.5
        )], key = lambda t: t.prevDay['v'], reverse=True)
        
        return [ticker.ticker for ticker in sort_lst]
    
    


In [48]:
'''Run test in Main form...'''
import os
import datetime as dt
#from portfolio_manager import PortfolioManager
ep = 'https://paper-api.alpaca.markets'
api_key = 'PK4LB1H4BH7YUF4XOODN'
secret_key = '7EuTmI/C/7FwtNkEzqzFD7yiVMqaI16XfuerzHgR'

os.environ['APCA_API_KEY_ID'] = api_key
os.environ['APCA_API_SECRET_KEY'] = secret_key

manager = PortfolioManager()
# Hedging SPY with GLD 1:1
'''
manager.add_items([
    ['SPY', 0.5],
    ['GLD', -0.005],
])
'''
print(f' ~~~ Percent Rebalance ~~~ ')
#manager.percent_rebalance('timeout') #TO RUN


Getting current ticker data...
All symbols loaded
 ~~~ Percent Rebalance ~~~ 


In [49]:
ETB = manager.get_tickers()

Getting current ticker data...
All symbols loaded


In [42]:
ETB[:10]
SE[:10]
full_se = SE
print(f'Pre Filter: Length of ETB: {len(ETB)}; Length of SE: {len(SE)}')
SE = list(set(SE).intersection(ETB))
print(f'Post Filter: Length of ETB: {len(ETB)}; Length of SE: {len(SE)}')

Pre Filter: Length of ETB: 2271; Length of SE: 999
Post Filter: Length of ETB: 2271; Length of SE: 0


In [36]:
'''Sort by volume'''
def SortFn(ticker):
    return ticker.prevDay['v']


#print sorted(strs, key=MyFn)  ## ['wa', 'zb', 'xc', 'yd']

#sort_by_volume = sorted(tickers,key = SortFn,reverse=True)
sort_by_volume = sorted((t for t in ETB),key=lambda t: t.prevDay['v'],reverse=True)

_SE = [ticker.ticker for ticker in sort_by_volume if ticker.ticker in SE]
#_SE = _SE[:100]
print(_SE)
len(_SE)

[]


0

In [77]:

'''Run w parse in main format'''

api_key = 'PK4LB1H4BH7YUF4XOODN'
secret_key = '7EuTmI/C/7FwtNkEzqzFD7yiVMqaI16XfuerzHgR'

os.environ['APCA_API_KEY_ID'] = api_key
os.environ['APCA_API_SECRET_KEY'] = secret_key
manager = PortfolioManager()

path = '/Users/zoakes/Desktop/rank/rankanalysis_05-07-2020.csv'
l,s = manager.parse(path)

arg = manager.format_holdings(l)
print(arg)
manager.add_items(arg)

#manager.percent_rebalance('timeout')

[['AAPL', 751.85992481203], ['ACLS', 751.85992481203], ['ADBE', 751.85992481203], ['ALNY', 751.85992481203], ['AMD', 751.85992481203], ['AMGN', 751.85992481203], ['AMSWA', 751.85992481203], ['AMZN', 751.85992481203], ['ASFI', 751.85992481203], ['ASML', 751.85992481203], ['ASTC', 751.85992481203], ['ATHX', 751.85992481203], ['ATRC', 751.85992481203], ['ATVI', 751.85992481203], ['AUBN', 751.85992481203], ['BANFP', 751.85992481203], ['BLDP', 751.85992481203], ['CDNS', 751.85992481203], ['CHTR', 751.85992481203], ['CJJD', 751.85992481203], ['CRUS', 751.85992481203], ['CTXS', 751.85992481203], ['CYBE', 751.85992481203], ['CYTK', 751.85992481203], ['DSGX', 751.85992481203], ['DSPG', 751.85992481203], ['EBAY', 751.85992481203], ['EGOV', 751.85992481203], ['ENTG', 751.85992481203], ['EQIX', 751.85992481203], ['ESEA', 751.85992481203], ['GLUU', 751.85992481203], ['HALO', 751.85992481203], ['HERO', 751.85992481203], ['HOLX', 751.85992481203], ['ICAD', 751.85992481203], ['IMMU', 751.85992481203],

In [ ]:
'''PROPER FORMAT!!'''
[['SPY', 0.5],['GLD', 0.5],]

In [124]:

def format_holdings(LONGS,SHORTS=[]):
    '''Returns proper format for PF Weights'''

    #longs = ['SPY','GLD','AMZN']
    #shorts = ['AAPL','TLT']
    print(len(longs),len(shorts))

    
    qty = 1.0 / (len(longs) + len(shorts)) 
    arg = []
    if len(longs) > 0:
        for l in longs:
            arg.append([l,qty])
            
    if len(shorts) > 0:
        for s in shorts:
            arg.append([s,-qty])
            

        
    return arg

#format_holdings()

In [39]:
    import pandas as pd


    def parse(path=None):
        #Vectorized operations
        
        #global LE, SE, CLOSE_DICT
        
        #old_long = LE
        #old_short = SE
        
        #new_long = set()
        #new_short = set()
        
        #new_lx = set()
        #new_sx = set()
        
        #DF = pd.DataFrame({'RA_T':[-1,3,5],'SYMBOL':['AAPL','INTC','AMD'],'P_Change':[1.5,2.5,3.5]})  #TEMPORARY !!!
        if path is not None:
            DF = pd.read_csv(path,header=None)
            DF.columns = ['Exchange','SYMBOL','RA_Y','RA_T','Close','P_Change']
            #DF = pd.read_csv("C:\\Users\\Administrator\\Downloads\\rankanalysis_05-05-2020.csv",header=None)
        
        #LE's
        strong = DF[DF['RA_T'] >= 5]
        mu = DF['P_Change'].mean()
        sig = DF['P_Change'].std()
        no_out = strong[strong['P_Change'] <= (mu+sig)]
        
        #SE's 
        weak = DF[DF['RA_T'] <= -5]
        #short_new = weak[weak['P_Change'] <= (mu+sig)]
        
        LE = no_out.SYMBOL.to_list()
        #LX = [i for i in old_long if i not in LE]
        
        SE = weak.SYMBOL.to_list()
        
        return LE , SE

LE, SE = parse(path = '/Users/zoakes/Desktop/rank/rankanalysis_05-08-2020.csv')
len(LE), len(SE)

(170, 894)

In [4]:
    '''IVE RAN THESE CELLS TOO MANY TIMES, NAMESPACE CORRUPT'''
    qty = 1.0 / (len(LE) + len(SE)) 
    arg = []
    if len(LE) > 0:
        for l in LE:
            arg.append([l,qty])
            
    if len(SE) > 0:
        for s in SE:
            arg.append([s,-qty])
   
arg[:10]

[['AAPL', 0.0008833922261484099],
 ['ACLS', 0.0008833922261484099],
 ['ADBE', 0.0008833922261484099],
 ['ALNY', 0.0008833922261484099],
 ['AMD', 0.0008833922261484099],
 ['AMGN', 0.0008833922261484099],
 ['AMSWA', 0.0008833922261484099],
 ['AMZN', 0.0008833922261484099],
 ['ASFI', 0.0008833922261484099],
 ['ASML', 0.0008833922261484099]]

In [9]:
'''RUNNING SUPER MINIMAL'''
manager.add_items(arg)

In [ ]:
print(f' ~~~ Percent Rebalance ~~~ ')
manager.percent_rebalance('timeout')

In [73]:
[['SPY',10]] + [['QQQ',-10]]

[['SPY', 10], ['QQQ', -10]]

In [4]:
manager.r_positions

{}

IndentationError: unexpected indent (<ipython-input-78-c314f65cefd8>, line 5)

In [82]:
qty =  -1
("buy", "sell")[qty < 0]

'sell'

In [79]:


def get_tickers(self,min_price=5,max_price=50,etb=True):
    print('Getting current ticker data...')
    tickers = self.api.polygon.all_tickers() #ALLLLLL SYMBOLS ! 
    print('All symbols loaded')
    assets = self.api.list_assets()
    if etb: #FILTER TRADABLE SYMBOLS; easy to borrow if switch True
        symbols = [asset.symbol for asset in assets if asset.tradable and asset.easy_to_borrow]
    else:
        symbols = [asset.symbol for asset in assets if asset.tradable] 
    #print(tickers)
    return [ticker.ticker for ticker in tickers if (
        ticker.ticker in symbols and
        ticker.lastTrade['p'] >= min_price and
        ticker.lastTrade['p'] <= max_price and
        ticker.prevDay['v'] * ticker.lastTrade['p'] > 500000 #and
        #ticker.todaysChangePerc >= 3.5
        #ticker.easy_to_borrow NOT IN TICKER
    )]


#tickers = get_tickers()
#tickers

In [77]:
'''confirm correct date in sheet!'''
path = '/Users/zoakes/Desktop/rankanalysis_05-09-2020.csv'
today = path[-14:-4]

now = dt.datetime.now()
now_st = now.strftime('%m-%d-%Y')
now_st

today == now_st
assert (path[-14:-4] == dt.datetime.now().strftime('%m-%d-%Y')) , 'CSV may be incorrect -- check that using curr csv'

In [91]:
assert today == now, 'CSV May be incorrect' 

NameError: name 'today' is not defined

In [95]:
import datetime as dt
now = dt.datetime.now()
now - dt.timedelta(days=1)

datetime.datetime(2020, 5, 11, 8, 16, 4, 179021)

In [19]:
def count():
    i = 0
    while 1:
        yield i
        i+= 1

it = count()


<generator object count at 0x107915e50>

In [26]:
def gen():
    i = 0
    while i < 10:
        yield i
        i += 1

it = gen()
it = iter(it)
lc = [list(iter(it)) for i in range(10)]
lc

it = gen()
#comp = [next(iter(it)) for i in range(10)]
list(it)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [17]:
lc = [next(iter(gen())) for i in range(10)]
lc

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [27]:
it = count()
lc = [next(iter(it)) for i in range(10)]
lc

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [41]:
LE = 
SE = 

In [43]:
path = '/Users/zoakes/Desktop/rank/rankanalysis_05-11-2020.csv'
df  = pd.read_csv(path,header=None)
df.columns = ['Exchange','SYMBOL','RA_Y','RA_T','Close','P_Change']

In [57]:
total_511 = 0
total_l = 0
total_s = 0
for i in df.itertuples():
    if i.SYMBOL in LE:
        total_l += 500 * (i.P_Change/100)
    elif i.SYMBOL in SE:
        total_s +=  -500 * (i.P_Change/100)

        
total_511 = (total_l + total_s)
print(f'Total LE {total_l}; Total SE {total_s} -- Total {total_511}')

Total LE 933.4; Total SE 1487.049999999999 -- Total 2420.449999999999


In [180]:
def calc_return(path,LE,SE,slip_usd=3,fill_pct=.8,usd=False):
    df = pd.read_csv(path,header=None)
    df.columns = ['Exchange','SYMBOL','RA_Y','RA_T','Close','P_Change']
    positions = len(LE) + len(SE)
    total_l, total_s = 0,0
    for i in df.itertuples():
        if i.SYMBOL in LE:
            total_l += 500 * (i.P_Change/100)
        elif i.SYMBOL in SE:
            total_s += -500 * (i.P_Change/100)
    
    total_usd = total_l + total_s
    
    real_usd = total_usd * fill_pct - (slip_usd * positions) 
    if usd:
        return real_usd 
    return (real_usd / 100000) * 100 

#calc_return(path,LE,SE,slip_usd=5,fill_pct=.65)
calc_return(path,LE,SE,slip_usd=4,fill_pct=1.0)

1.5730999999999995

In [187]:
LE = ['AMD', 'AAPL', 'QQQ', 'MSFT', 'EBAY', 'ATVI', 'NVDA', 'TLT', 'SE', 'ATRC', 'DXCM', 'IMMU', 'TAL', 'NEM', 'DT', 'HALO', 'LLY', 'NFLX', 'HD', 'AMGN', 'ADBE', 'GNMK', 'CTXS', 'FTNT', 'CYTK', 'HOLX', 'CGEN', 'CDNS', 'AU', 'NUAN', 'LOGI', 'SHY', 'QDEL', 'ENTG', 'CNST', 'DG', 'HRL', 'VGSH', 'TREX', 'LSCC', 'VRTX', 'ACLS', 'ALNY', 'AUDC', 'OSUR', 'TEAM', 'CRUS', 'FLWS', 'SGEN', 'RBA', 'CPB', 'AEM', 'CLX', 'MNTA', 'HUM', 'JKHY', 'VGIT', 'VMBS', 'ASML', 'BKI', 'LMNX', 'MASI', 'FORM', 'DPZ', 'TBIO', 'DVA', 'STMP', 'PZZA', 'WERN', 'SMG', 'EBS', 'WGO', 'PETS', 'RGLD', 'SNPS', 'CALX', 'CCC', 'MSCI', 'NICE', 'DSPG', 'SLGN', 'WST', 'NTES', 'RGEN', 'MOH', 'RDY', 'CRL', 'MPWR', 'TECH', 'AMSWA', 'TYL', 'PODD', 'DSGX', 'SPNS', 'PEGA', 'SPAR', 'TECD', 'CYBE', 'POWI', 'MRCY']
SE = ['T', 'WFC', 'OXY', 'LUV', 'SPG', 'C', 'SLB', 'MGM', 'USB', 'EPD', 'CNP', 'FITB', 'OKE', 'AGNC', 'LVS', 'PXD', 'GM', 'JCI', 'IR', 'SU', 'KSS', 'EOG', 'STWD', 'PPL', 'BP', 'HPQ', 'CVX', 'DVN', 'RDS.A', 'GLW', 'RDS.B', 'MET', 'EQR', 'COP', 'CTSH', 'DISCK', 'CAKE', 'UPS', 'SPR', 'VTR', 'FNF', 'AES', 'EXPE', 'BK', 'MUR', 'IRM', 'EMR', 'DOW', 'AIG', 'UNM', 'SIX', 'OHI', 'WDC', 'COF', 'TOL', 'PRU', 'WY', 'LNC', 'ACWX', 'BLUE', 'TD', 'HOG', 'CLR', 'MFC', 'FL', 'ARCC', 'RL', 'MOS', 'SLG', 'CNQ', 'WYNN', 'DD', 'KAR', 'DFS', 'SNV', 'TRV', 'BEN', 'ACC', 'NUE', 'AFL', 'RDN', 'GEO', 'ORI', 'GS', 'NOV', 'GIL', 'EPR', 'CB', 'HXL', 'VNO', 'FLS', 'XRAY', 'PPG', 'OMC', 'L', 'LEG', 'TOT', 'UGI', 'TU', 'EAT']

new_path = '/Users/zoakes/Desktop/rank/rankanalysis_05-13-2020.csv'
calc_return(new_path,LE,SE,2.5,1.0)

1.3379499999999993

In [ ]:
for row, index in df.iterrows():
    #print(row,index)
    if row > 0:
        past = df.iloc[row-1]
        print(past.P_Change)
    print(index.SYMBOL)

In [221]:
import pandas_datareader as pdr

aapl_qqq = pdr.DataReader(['SPY','QQQ'],'yahoo',2020)

In [222]:
(aapl_qqq.pct_change() * 100).tail(1)

Attributes Adj Close               Close                High            \
Symbols          SPY       QQQ       SPY       QQQ       SPY       QQQ   
Date                                                                     
2020-05-15  0.463209  0.649147  0.463209  0.649147  0.427905  0.658046   

Attributes       Low               Open               Volume            
Symbols          SPY       QQQ      SPY       QQQ        SPY       QQQ  
Date                                                                    
2020-05-15  3.058722  1.088009  1.22602  0.367024 -21.292727  1.906274

In [113]:
dt.datetime.today()

dct = {}
dct[dt.datetime.today().strftime('%m-%d-%Y')] = 1
dct

{'05-12-2020': 1}

In [117]:
LE = ['MSFT', 'AAPL', 'QQQ', 'EBAY', 'INTC', 'TLT', 'NFLX', 'DPZ', 'SE', 'FAST', 'NEM', 'GIS', 'GLUU', 'TIF', 'HD', 'SWKS', 'SHY', 'TAL', 'CGEN', 'AU', 'IMMU', 'CTXS', 'LSCC', 'FTNT', 'TEAM', 'ADBE', 'ENTG', 'AMGN', 'DLR', 'VMBS', 'CDNS', 'TREX', 'CCI', 'HOLX', 'AEM', 'VGSH', 'SGEN', 'ROK', 'HRL', 'ACAD', 'CLX', 'CRUS', 'AUDC', 'CPB', 'CATS', 'EGOV', 'VGIT', 'MPWR', 'PZZA', 'SMG', 'KLAC', 'CNST', 'RBA', 'TBIO', 'BKI', 'TYL', 'NTES', 'WGO', 'ACLS', 'PETS', 'DVA', 'SNPS', 'FLWS', 'ATRC', 'CCC', 'VGLT', 'SPNS', 'RMBS', 'PEGA', 'HUM', 'TECD', 'MRCY', 'FORM', 'RGLD', 'SLGN', 'NDSN', 'DSPG', 'RDY', 'MOH', 'ASML', 'CYBE', 'JKHY', 'VICR', 'SPAR', 'NICE', 'MRTN', 'POOL', 'AXE', 'CRL', 'CALX', 'FRPT', 'DSGX', 'PDFS', 'POWI', 'NVMI', 'STN', 'SAIA', 'BHK', 'AMSWA', 'AHPI'] 
 
SE = ['T', 'WFC', 'OXY', 'LUV', 'C', 'SLB', 'XOM', 'MGM', 'BA', 'SPG', 'HAL', 'KEY', 'GM', 'KSS', 'HST', 'USB', 'EPD', 'LVS', 'EOG', 'WDC', 'HPQ', 'SU', 'WYNN', 'CNP', 'SPR', 'PPL', 'STWD', 'RDS.A', 'MFC', 'JCI', 'AIG', 'DVN', 'PXD', 'EMR', 'OKE', 'SNV', 'CVX', 'PBF', 'RDS.B', 'TD', 'BP', 'COP', 'MET', 'OHI', 'STL', 'GLW', 'AGNC', 'FNF', 'UPS', 'CLR', 'MOS', 'FLS', 'XRAY', 'IR', 'GEO', 'FITB', 'BWA', 'UNM', 'AES', 'ORI', 'BK', 'SIX', 'WY', 'KAR', 'DISCK', 'EQR', 'PEB', 'DD', 'IRM', 'NUE', 'TOL', 'UGI', 'JEF', 'MYGN', 'DOW', 'ZION', 'UMPQ', 'LNC', 'PRU', 'DFS', 'PVH', 'OGE', 'CNK', 'COF', 'BEN', 'TOT', 'VTR', 'FL', 'HOG', 'EXPE', 'MUR', 'EPR', 'URBN', 'PBCT', 'CB', 'OMC', 'NTAP', 'TXT', 'HP', 'LEG']

In [130]:
path = '/Users/zoakes/Desktop/rank/rankanalysis_05-11-2020.csv'
calc_return(path,LE,SE,slip_usd=3.5,fill_pct=200/200)

0.901449999999999

In [132]:
3/100000

3e-05

In [144]:
'''simulated Capacity testing'''
worst_stock = SE[-1]
print(worst_stock)
worst_stock = 'LEG'

omega = pdr.DataReader(worst_stock,'yahoo',2020)
omega.tail(1)

omega['ADV'] = omega.Volume.rolling(window=20).mean()

o_vol = omega.ADV.iloc[-1]
o_vol

LEG


2150567.65

In [161]:
avg_price = 135
shares_per_100k = 500 / avg_price
pct_of_adv = .0025 #2.5%

max_shares = pct_of_adv * o_vol
multiples_of_100k = max_shares / shares_per_100k #Multiple of scale larger than current, ex 2 would be 200k max

capacity_in_usd = multiples_of_100k * 100000 * 200 #*100k to get to USD, * 200 bc 200 symbols each day
capacity_in_usd
print(capacity_in_usd / 1e9) #29B

29.032663274999994


In [188]:
'''Covariance Testing -- Beta'''

LE =  ['SPY','QQQ', 'MSFT', 'AMD', 'AAPL', 'NVDA', 'NFLX', 'GIS', 'EBAY', 'SE', 'ATVI', 'TLT', 'SHY', 'NEM', 'IMMU', 'TAL', 'LLY', 'HOLX', 'QDEL', 'LOGI', 'DT', 'HALO', 'GNMK', 'AU', 'OSUR', 'FTNT', 'CDNS', 'SGEN', 'MNTA', 'HRL', 'DG', 'AMGN', 'FLWS', 'CYTK', 'DXCM', 'NBIX', 'TREX', 'NTES', 'PETS', 'CTXS', 'SLGN', 'CGEN', 'VRTX', 'SNPS', 'TEAM', 'CPB', 'CLX', 'LMNX', 'ATRC', 'ENTG', 'VGSH', 'MASI', 'RGLD', 'PZZA', 'AEM', 'DVA', 'ASML', 'DPZ', 'HUM', 'TBIO', 'MPWR', 'NBR', 'BKI', 'JKHY', 'CRUS', 'CCC', 'WGO', 'VMBS', 'ALNY', 'RGEN', 'ACMR', 'WERN', 'EBS', 'MRCY', 'SMG', 'CNST', 'MSCI', 'AUDC', 'VGIT', 'STMP', 'TECD', 'WST', 'CRL', 'MOH', 'TYL', 'TECH', 'CALX', 'MRTN', 'POWI', 'AHPI', 'AXE', 'DSPG', 'NVMI', 'AMSWA', 'CYBE', 'VGLT', 'VICR', 'WMK', 'APEI', 'IGI', 'HERO'] 
import pandas_datareader as pdr

LE_CVAR = pdr.DataReader(LE,'yahoo','2020')

In [198]:
SE = ['SPY','T', 'WFC', 'OXY', 'LUV', 'MGM', 'C', 'SLB', 'USB', 'SPG', 'FITB', 'EPD', 'GM', 'OKE', 'KSS', 'RDS.A', 'TD', 'PPL', 'RDS.B', 'LVS', 'AGNC', 'HPQ', 'JCI', 'CVX', 'STWD', 'EOG', 'SPR', 'AXP', 'SU', 'BP', 'IR', 'DVN', 'AES', 'GLW', 'PBCT', 'WY', 'MUR', 'CNP', 'SIX', 'MET', 'SLG', 'UPS', 'EPR', 'EMR', 'COP', 'PXD', 'PRU', 'EQR', 'WYNN', 'ORI', 'BK', 'RDN', 'JEF', 'UNM', 'IRM', 'WDC', 'AIG', 'SNV', 'IBM', 'DOW', 'GIL', 'FDX', 'CTSH', 'BEN', 'DFS', 'OHI', 'FNF', 'O', 'CAKE', 'NUE', 'IPG', 'MOS', 'FL', 'VTR', 'COF', 'MFC', 'OMC', 'DKS', 'SBGI', 'GEO', 'L', 'MMM', 'STT', 'TOT', 'TOL', 'WU', 'LNC', 'MIDD', 'VNO', 'EXPE', 'DD', 'AFL', 'CNQ', 'HOG', 'LEG', 'KAR', 'SWK', 'EAT', 'FLS', 'CNK', 'NTAP']

SE_CVAR = pdr.DataReader(SE,'yahoo',2020)

/Users/zoakes/opt/anaconda3/lib/python3.7/site-packages/pandas_datareader/base.py:270: SymbolWarning: Failed to read symbol: 'RDS.A', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/zoakes/opt/anaconda3/lib/python3.7/site-packages/pandas_datareader/base.py:270: SymbolWarning: Failed to read symbol: 'RDS.B', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)


In [204]:
SCV = SE_CVAR.pct_change().cov()
short_total = SCV.loc['Adj Close','SPY'].sum() * -1
short_total 

-0.3929580784926588

In [206]:
LCV = LE_CVAR.pct_change().cov()
long_total = LCV.loc['Adj Close','SPY'].sum()


long_total

-0.00277036805300529

In [215]:




def estimate_beta(le=None,se=None,agg='S'):
    if le or se is None:
        global LE, SE
        le = LE
        se = SE
    if 'SPY' not in le:
        le.append('SPY')
    if 'SPY' not in se:
        se.append('SPY')
    
    LECV = pdr.DataReader(le,'yahoo',2020)
    LCV = LECV.pct_change().cov()

    SECV = pdr.DataReader(se,'yahoo',2020)
    SCV = SECV.pct_change().cov()
    
    if agg == 'S':
        long_total = LCV.loc['Adj Close','SPY'].sum()
        short_total = SCV.loc['Adj Close','SPY'].sum()
    else:
        long_total = LCV.loc['Adj Close','SPY'].mean()
        short_total = SCV.loc['Adj Close','SPY'].mean()
    short_total *= -1
    
    return long_total + short_total
    
    
    
#estimate_beta()   

In [216]:
'''5.15.20'''
LE = ['QQQ', 'MSFT', 'AMD', 'AAPL', 'NVDA', 'NFLX', 'GIS', 'EBAY', 'SE', 'ATVI', 'TLT', 'SHY', 'NEM', 'IMMU', 'HD', 'TAL', 'LLY', 'HOLX', 'QDEL', 'LOGI', 'DT', 'KLAC', 'HALO', 'GNMK', 'AU', 'OSUR', 'FTNT', 'CDNS', 'SGEN', 'MNTA', 'DG', 'AMGN', 'FLWS', 'CYTK', 'DXCM', 'TREX', 'NTES', 'PETS', 'CTXS', 'SLGN', 'CGEN', 'VRTX', 'SNPS', 'TEAM', 'CPB', 'CLX', 'LMNX', 'ATRC', 'ENTG', 'VGSH', 'MASI', 'RGLD', 'PZZA', 'AEM', 'ASML', 'FORM', 'DPZ', 'HUM', 'TBIO', 'MPWR', 'NBR', 'MKC', 'BKI', 'JKHY', 'CRUS', 'CCC', 'WGO', 'VMBS', 'ALNY', 'RGEN', 'ACMR', 'NDSN', 'WERN', 'EBS', 'SMG', 'CNST', 'MSCI', 'AUDC', 'VGIT', 'STMP', 'TECD', 'WST', 'CRL', 'MOH', 'TYL', 'TECH', 'CALX', 'MRTN', 'POWI', 'AHPI', 'SAIA', 'AXE', 'DSPG', 'NVMI', 'AMSWA', 'CYBE', 'VGLT', 'VICR', 'WMK', 'DSGX'] 
SE = ['T', 'WFC', 'OXY', 'LUV', 'MGM', 'C', 'SLB', 'USB', 'SPG', 'FITB', 'HAL', 'HST', 'EPD', 'GM', 'OKE', 'KSS', 'RDS.A', 'TD', 'PPL', 'RDS.B', 'LVS', 'AGNC', 'HPQ', 'JCI', 'CVX', 'STWD', 'EOG', 'SPR', 'SU', 'BP', 'IR', 'DVN', 'AES', 'GLW', 'PBCT', 'WY', 'MUR', 'CNP', 'SIX', 'MET', 'SLG', 'UPS', 'EPR', 'EMR', 'COP', 'PXD', 'PRU', 'EQR', 'WYNN', 'ORI', 'BK', 'JEF', 'UNM', 'IRM', 'WDC', 'AIG', 'SNV', 'IBM', 'DOW', 'GIL', 'FDX', 'CTSH', 'BEN', 'DFS', 'OHI', 'FNF', 'O', 'CAKE', 'NUE', 'IPG', 'MOS', 'FL', 'VTR', 'COF', 'STL', 'MFC', 'OMC', 'DKS', 'SBGI', 'GEO', 'L', 'MMM', 'STT', 'TOT', 'TOL', 'WU', 'LNC', 'MIDD', 'VNO', 'EXPE', 'DD', 'AFL', 'CNQ', 'HOG', 'LEG', 'KAR', 'SWK', 'EAT', 'FLS', 'CNK']

In [218]:
estimate_beta(LE,SE,'M')

-0.00037377431830152614

In [261]:
'''TO GET POINT IN TIME VOLUME'''

sq = pdr.DataReader(['SPY','QQQ'],'yahoo',2010) #.Volume #.head(1) #.sort_values(by='Symbols')
vol = sq.Volume
jan410 = sq.Volume.sort_values(by='2010-01-04',axis=1)
print(jan410.columns)

dates = []
for row, index in sq.iterrows():
    dates.append(row)
    #tmp = row.Volume.sort_values(by=index,axis=1)

Index(['QQQ', 'SPY'], dtype='object', name='Symbols')


In [285]:
dates #Rows!
from collections import defaultdict

values = defaultdict(list)
for d in dates:#range(df.shape[0]):
    tmp = vol.sort_values(by=d,axis=1)
    #print(tmp.columns.values)
    values[d].append(tmp.columns.values)
    
values

defaultdict(list,
            {Timestamp('2010-01-04 00:00:00'): [array(['QQQ', 'SPY'], dtype=object)],
             Timestamp('2010-01-05 00:00:00'): [array(['QQQ', 'SPY'], dtype=object)],
             Timestamp('2010-01-06 00:00:00'): [array(['QQQ', 'SPY'], dtype=object)],
             Timestamp('2010-01-07 00:00:00'): [array(['QQQ', 'SPY'], dtype=object)],
             Timestamp('2010-01-08 00:00:00'): [array(['QQQ', 'SPY'], dtype=object)],
             Timestamp('2010-01-11 00:00:00'): [array(['QQQ', 'SPY'], dtype=object)],
             Timestamp('2010-01-12 00:00:00'): [array(['QQQ', 'SPY'], dtype=object)],
             Timestamp('2010-01-13 00:00:00'): [array(['QQQ', 'SPY'], dtype=object)],
             Timestamp('2010-01-14 00:00:00'): [array(['QQQ', 'SPY'], dtype=object)],
             Timestamp('2010-01-15 00:00:00'): [array(['QQQ', 'SPY'], dtype=object)],
             Timestamp('2010-01-19 00:00:00'): [array(['QQQ', 'SPY'], dtype=object)],
             Timestamp('2010-01-20 0